In [ ]:
from IPython import get_ipython
from IPython.display import display
# %%
import os
import numpy as np

from google.colab import drive
drive.mount('/content/drive')

from tensorflow import keras
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, classification_report
from PIL import Image
from PIL import UnidentifiedImageError

# 1. Definir diretórios e parâmetros
train_dir = '/content/drive/MyDrive/_Treino_transfer/PetImages'     # Diretório de treino
val_dir =   '/content/drive/MyDrive/_Treino_transfer/validation'    # Diretório de validação
test_dir =  '/content/drive/MyDrive/_Treino_transfer/test'          # Diretório de teste
img_height, img_width = 224, 224  # Dimensões das imagens
batch_size = 32  # Tamanho do lote
epochs = 1  # Número de épocas de treinamento

# 2. Verificar existência dos diretórios
if not os.path.exists(train_dir): raise FileNotFoundError(f"Diretório de treino não encontrado: {train_dir}")
if not os.path.exists(val_dir):   raise FileNotFoundError(f"Diretório de validação não encontrado: {val_dir}")
if not os.path.exists(test_dir):  raise FileNotFoundError(f"Diretório de teste não encontrado: {test_dir}")

# Function to check if a file is a valid image
def is_valid_image(filepath):
    """Checks if a file is a valid image."""
    try:
        Image.open(filepath).verify()  # Attempt to open and verify the image
        return True
    except (IOError, SyntaxError, ValueError, UnidentifiedImageError) as e:  # Catch potential image errors including UnidentifiedImageError
        print(f"Error with image file: {filepath}. Error: {e}")  # Print the error for debugging
        return False

# Function to filter out invalid image files
def filter_invalid_images(directory):
    """Filters out invalid image files from a directory."""
    for filename in os.listdir(directory):
        filepath = os.path.join(directory, filename)
        # Check if it's a file before attempting to remove
        if os.path.isfile(filepath) and not is_valid_image(filepath):
            print(f"Invalid image file found and removed: {filepath}")
            os.remove(filepath)

# Filter out invalid images from train, validation, and test directories
for root, _, files in os.walk(train_dir):
    for file in files:
        # Construct the full file path
        filepath = os.path.join(root, file)
        # Call is_valid_image directly on the file path
        if not is_valid_image(filepath):
            print(f"Invalid image file found and removed: {filepath}")
            os.remove(filepath)
for root, _, files in os.walk(val_dir):
    for file in files:
        # Construct the full file path
        filepath = os.path.join(root, file)
        # Call is_valid_image directly on the file path
        if not is_valid_image(filepath):
            print(f"Invalid image file found and removed: {filepath}")
            os.remove(filepath)
for root, _, files in os.walk(test_dir):
    for file in files:
        # Construct the full file path
        filepath = os.path.join(root, file)
        # Call is_valid_image directly on the file path
        if not is_valid_image(filepath):
            print(f"Invalid image file found and removed: {filepath}")
            os.remove(filepath)


# 3. Carregar modelo ResNet50 pré-treinado
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# 4. Congelar camadas iniciais
for layer in base_model.layers:
    layer.trainable = False

# 5. Adicionar camadas personalizadas
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(2, activation='softmax')(x)  # 2 classes: cães e gatos

model = Model(inputs=base_model.input, outputs=predictions)

# 6. Compilar modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 7. Geradores de dados
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)  # Gerador para dados de teste

train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

val_data = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

# 8. Treinar modelo
history = model.fit(
    train_data,
    epochs=epochs,
    validation_data=val_data
)

# 9. Avaliar modelo nos dados de teste
loss, accuracy = model.evaluate(test_data)
print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')

# 10. Matriz de confusão e relatório de classificação
predictions = model.predict(test_data)
predicted_labels = np.argmax(predictions, axis=1)
true_labels = test_data.classes

cm = confusion_matrix(true_labels, predicted_labels)
print('Matriz de Confusão:\n', cm)

cr = classification_report(true_labels, predicted_labels)
print('Relatório de Classificação:\n', cr)

# 11. (Opcional) Salvar o modelo treinado
model.save('/content/drive/MyDrive/_Treino_transfer/modelo_cachorro_gato.h5')